<div class="alert alert-block alert-info">
<h1 style="text-align:center;font-family:avenir;">GLM first level</h1>
Lou Planchamp & Victor Férat
</div>

In [ ]:
%matplotlib inline

In [ ]:
from nilearn import plotting
from nilearn.plotting import plot_stat_map
import pandas as pd
from nilearn.glm.first_level import FirstLevelModel
from numpy import array
import os
from nilearn.plotting import plot_design_matrix
import numpy as np
import matplotlib.pyplot as plt

The following function aim to specify the experimental paradigm

We provide a description of the experiment, that is, define the timing of the different blocks of the experiment

This is provided in an events.tsv file

There are one function for each cognitive task Calcul, Memory, CPT & for the Transfer run done only in session 11


In [ ]:
# We need this function to select the lines that begins with "Bloc" and has no "Trial" define
# We use it to create the variable "test" few lines later...
def is_bloc(x):
    x = str(x)
    return(not ('Trial' in x) and (x.startswith('Bloc')))

def read_trials_calcul(events_filename):
    
    events_file = pd.read_csv(events_filename)
    assert events_file['Event'][1] == '[MRI_Sync]'
    t0 = events_file['TimeStamp'][1]
    
    events_file['t'] = (events_file['TimeStamp'] - t0) / 1000
    events_file['test'] = events_file['Unnamed: 3'].apply(lambda x: is_bloc(x))
    
    # Define onsets, durations and trial_type
    events_starts = events_file[events_file['test'] == True][['t', 'Unnamed: 4']]
    events_stops = events_file[(events_file['Unnamed: 3'].str.endswith("Trial_4")) & (events_file['Unnamed: 4'] == 'Trial Finished')][['Unnamed: 4', 't']]
    
    assert events_starts['Unnamed: 4'][3] == 'Calcul'
    
    durations = events_stops['t'].values - events_starts['t'].values
    onsets = events_starts['t']
    trial_types = events_starts['Unnamed: 4']
    
    events = pd.DataFrame()
    events['onset'] = onsets
    events['duration'] = durations
    events['trial_type'] = trial_types
    
    assert (events['duration'].all() <= 30.2 and events['duration'].any() <= 29.8 )
    
    return(events)

#===========================================================================

def read_trials_memory(events_filename):
    
    events_file = pd.read_csv(events_filename)
    assert events_file['Event'][1] == '[MRI_Sync]'
    t0 = events_file['TimeStamp'][1]
    
    events_file['t'] = (events_file['TimeStamp'] - t0) / 1000
        
    # Define onsets, durations and trial_type
    events_starts = events_file[(events_file['Unnamed: 5'] == 'Stimuli presented')][['t', 'Unnamed: 4', 'Unnamed: 5']]
    events_stops = events_file[(events_file['Unnamed: 5'] == 'White Screen presented')][['t', 'Unnamed: 5']]
    
    assert events_starts['Unnamed: 4'][4] == 'First'
    
#    durations = events_stops['t'].values - events_starts['t'].values
    durations = 0.0
    onsets = events_starts['t']
    trial_types = events_starts['Unnamed: 4']
    
    events = pd.DataFrame()
    events['onset'] = onsets
    events['duration'] = durations
    events['trial_type'] = trial_types
    
#    assert (events['duration'].all() <= 4.2 and events['duration'].any() <= 3.8 )
    
    return(events)

#===========================================================================

def read_trials_cpt(events_filename):
    
    try:
    # session 2    
        # Load t0
        events_start = pd.read_csv(events_filename, sep=';', header=1, nrows=1, names=['Timestamps', 'EventID', 'FPS', 'value', 'misc'])
        t0 = events_start['Timestamps'][0]

        # Load table
        read_events = pd.read_csv(events_filename,
                             sep=';',
                             header=0,
                             skiprows=4,
                             names=['Timestamps', 'EventID', 'Event', 'Bloc', 'info', 'stim', 'resp', 'misc'])

        read_events['t'] = (read_events['Timestamps'] - t0) / 1000
        
    except Exception as e:
    # session 11 
        print(e)
        # Load t0
        events_start = pd.read_csv(events_filename, sep=';', header=2, nrows=1, names=['Timestamps', 'EventID', 'FPS', 'value', 'misc'])
        t0 = events_start['Timestamps'][0]

        # Load table
        read_events = pd.read_csv(events_filename,
                             sep=';',
                             header=0,
                             skiprows=5,
                             names=['Timestamps', 'EventID', 'Event', 'Bloc', 'info', 'stim', 'resp', 'misc'])

        read_events['t'] = (read_events['Timestamps'] - t0) / 1000
        
        
    # Define onsets, durations and trial_type
    events_starts = read_events[(read_events['Bloc'].str.endswith("Trial_0")) & (read_events['stim'] == 'Fixation presented')][['Bloc', 't']]
    events_stops = read_events[(read_events['Bloc'].str.endswith("Trial_19")) & (read_events['info'] == 'Trial Finished')][['Bloc', 't']]

    durations = events_stops['t'].values - events_starts['t'].values
    onsets = events_starts['t'].values

    trial_types = []
    for name in events_starts['Bloc'].values:
        is_letter = int(name.split('_')[1]) % 2 == 0
        if is_letter:
            trial_types.append('letter')
        else:
            trial_types.append('symbol')

    events = pd.DataFrame()
    events['onset'] = onsets
    events['duration'] = durations
    events['trial_type'] = trial_types
    
    assert (events['duration'].all() <= 40.5 and events['duration'].any() <= 39.8 )
    
    return(events)

#===========================================================================

def read_trials_transfer(events_filename):
    events = pd.DataFrame()
    events['onset'] = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]
    events['duration'] = [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
    events['trial_type'] = ['regu', 'rest','regu', 'rest','regu', 'rest','regu', 'rest','regu', 'rest','regu', 'rest']
    return(events)


The following function is to define which of the preceding function to use depending on the task 

In [ ]:
def read_trials(events_filename, task):
    if task == 'Calcul':
        events  = read_trials_calcul(events_filename)
        return(events)
    if task == 'CPT':
        events  = read_trials_cpt(events_filename)
        return(events)
    if task == 'Memory':
        events  = read_trials_memory(events_filename)
        return(events)
    if task == 'Transfer':
        events  = read_trials_transfer(events_filename)
        return(events)


    
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

#fig,ax = render_mpl_table(df, header_columns=0, col_width=4.0)
#fig.savefig(f'first_level_GLM/design_matrix/events/sub-{subject}_ses-{session}_{task}_event.png')
    

Both of these functions use the FirstLevelModel of nilearn

It will generate the design matrix using the information provided by the events object

In [ ]:
def compute_z_map(fmri_img, events_filename, confounds_filename):
    events = read_trials(events_filename, task)
    
    # Specify the parameters of the first-level model (just for the z-map)
    fmri_glm = FirstLevelModel(t_r=1.76,
                               noise_model='ar1',
                               standardize=False,
                               hrf_model='spm',
                               drift_model='cosine',
                               high_pass=.01,
                               smoothing_fwhm=10)
   
    # Specity the confounds (same for z and t)
    confounds = pd.read_table(confounds_filename)
    keep_confounds = ['csf', 
                      'white_matter', 
                      'trans_x', 'trans_x_derivative1', 'trans_x_derivative1_power2', 
                      'trans_y', 'trans_y_derivative1', 'trans_y_derivative1_power2', 
                      'trans_z', 'trans_z_derivative1', 'trans_z_derivative1_power2', 
                      'rot_x', 'rot_x_derivative1', 'rot_x_derivative1_power2', 
                      'rot_y', 'rot_y_derivative1', 'rot_y_derivative1_power2', 
                      'rot_z', 'rot_z_derivative1', 'rot_z_derivative1_power2'
                     ]
    nuisance_regs = confounds[keep_confounds]
    df = pd.DataFrame(nuisance_regs)
    nuisance_regs = df.fillna(0)

    fmri_glm = fmri_glm.fit(fmri_img, events, confounds=nuisance_regs)
    design_matrix = fmri_glm.design_matrices_[0]
    
    # To save a figure of the design matrix
    # Per task, just ad sub-{subject}_ses-{session} in name file to have all the different matrices 
    # Same for z and t so I just put it once
    plot_design_matrix(design_matrix,
                      output_file=f'first_level_GLM/design_matrix/{task}_design_matrix.png')

    condition1 = [0]*len(design_matrix.columns)
    condition1[0] = 1

    condition2 = [0]*len(design_matrix.columns)
    condition2[1] = 1

    conditions = {
    'first': array(condition1),
    'second':  array(condition2),
    }
 
    first_minus_second = conditions['first'] - conditions['second']


    z_map = fmri_glm.compute_contrast(first_minus_second,
                                     output_type='z_score')

    
    return(z_map, fmri_glm)

#===========================================================================

def compute_t_map(fmri_img, events_filename, confounds_filename):
    events = read_trials(events_filename, task)
    
    # Specify the parameters of the first-level model (just for the t-map)
    fmri_glm = FirstLevelModel(t_r=1.76,
                               noise_model='ar1',
                               standardize=False,
                               hrf_model='spm',
                               drift_model='cosine',
                               high_pass=.01,
                               smoothing_fwhm=5)
   
    # Specity the confounds (same for z and t)
    confounds = pd.read_table(confounds_filename)
    keep_confounds = ['csf', 
                      'white_matter', 
                      'trans_x', 'trans_x_derivative1', 'trans_x_derivative1_power2', 
                      'trans_y', 'trans_y_derivative1', 'trans_y_derivative1_power2', 
                      'trans_z', 'trans_z_derivative1', 'trans_z_derivative1_power2', 
                      'rot_x', 'rot_x_derivative1', 'rot_x_derivative1_power2', 
                      'rot_y', 'rot_y_derivative1', 'rot_y_derivative1_power2', 
                      'rot_z', 'rot_z_derivative1', 'rot_z_derivative1_power2'
                     ]
    nuisance_regs = confounds[keep_confounds]
    df = pd.DataFrame(nuisance_regs)
    nuisance_regs = df.fillna(0)

    fmri_glm = fmri_glm.fit(fmri_img, events, confounds=nuisance_regs)
    design_matrix = fmri_glm.design_matrices_[0]

    condition1 = [0]*len(design_matrix.columns)
    condition1[0] = 1

    condition2 = [0]*len(design_matrix.columns)
    condition2[1] = 1

    conditions = {
    'first': array(condition1),
    'second': array(condition2),
    }
 
    first_minus_second = conditions['first'] - conditions['second']
    
    t_map = fmri_glm.compute_contrast(first_minus_second,
                                      stat_type ='t',
                                      output_type='stat')
    
    return(t_map, fmri_glm)

This is the loop that will apply all the functions defined for all tasks, all subjects, all sessions

Results are saved in .nii

In [ ]:
if not os.path.isdir('first_level_GLM'):
    os.mkdir('first_level_GLM')
if not os.path.isdir('first_level_GLM/tmaps'):  
    os.mkdir('first_level_GLM/tmaps')
if not os.path.isdir('first_level_GLM/zmaps'): 
    os.mkdir('first_level_GLM/zmaps')
#'Transfer', 'Calcul', 'CPT', 
for task in ['Memory']:
    tfolder = f'first_level_GLM/tmaps/{task}'
    if not os.path.isdir(tfolder): 
        os.mkdir(tfolder)
    zfolder = f'first_level_GLM/zmaps/{task}'
    if not os.path.isdir(zfolder): 
        os.mkdir(zfolder)
    for subject in ['01', '02', '05', '07', '08']:
        for session in ['02', '11']:
            if task == 'Transfer':
                session = '11'
                if subject == '07':
                    subject = '05'
                # Not ideal because run twice but it works, just takes a bit more time 
                # To be modify
                
            print(task, subject, session)
            
            anat = f'/home/carole/Bureau/BIDS_fMRIprep/derivatives/sub-{subject}/ses-02/anat/sub-{subject}_ses-02_space-MNIPediatricAsym_cohort-3_desc-preproc_T1w.nii.gz'
            fmri_img = f'/home/carole/Bureau/BIDS_fMRIprep/derivatives/sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-{task.lower()}_space-MNIPediatricAsym_cohort-3_desc-preproc_bold.nii.gz'
            events_folder = f'/home/carole/Bureau/triggers/sub-{subject}_ses-{session}'
            
            for file in os.listdir(events_folder):
                    if file.startswith(f'MRI_{task}_Normal'):
                        events_filename = os.path.join(events_folder, file)
        
            if task == 'Transfer':
                for file in os.listdir(events_folder):
                    if file.startswith(f'MRI_{task}_Normal'):
                        events_filename = os.path.join(events_folder, file)
                    else :
                        events_filename = os.path.join(events_folder, 'jsp')
            confounds_filename = f"/home/carole/Bureau/BIDS_fMRIprep/derivatives/sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-{task.lower()}_desc-confounds_timeseries.tsv"

            t_map, fmri_glm = compute_t_map(fmri_img, events_filename, confounds_filename)
            tpath = os.path.join(tfolder, f'sub-{subject}_ses-{session}_{task}_tmap.nii')
            t_map.to_filename(tpath)
            
            z_map, fmri_glm = compute_z_map(fmri_img, events_filename, confounds_filename)
            zpath = os.path.join(zfolder, f'sub-{subject}_ses-{session}_{task}_zmap.nii')
            z_map.to_filename(zpath)
            
            df= read_trials(events_filename, task)
            fig,ax = render_mpl_table(df, header_columns=0, col_width=4.0)
            fig.savefig(f'first_level_GLM/design_matrix/events/sub-{subject}_ses-{session}_{task}_event.png')
